## Libraries
    Importing all the necessary libraries

In [1]:
import requests 
from urllib.parse import quote,unquote
import urllib.parse
import re
import unidecode
import pandas as pd


## Text Normalization 
    Cleaning the text from emojis and other unicode icons

    link to the original thread >> https://gist.github.com/slowkow/7a7f61f495e3dbb7e3d767f97bd7304b

In [2]:
def clean_data(data):
    emoj = re.compile("["
                u"\U0001F600-\U0001F64F"  # emoticons
                u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                u"\U0001F680-\U0001F6FF"  # transport & map symbols
                u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                u"\U00002500-\U00002BEF"  # chinese char
                u"\U00002702-\U000027B0"
                u"\U00002702-\U000027B0"
                u"\U000024C2-\U0001F251"
                u"\U0001f926-\U0001f937"
                u"\U00010000-\U0010ffff"
                u"\u2640-\u2642" 
                u"\u2600-\u2B55"
                u"\u200d"
                u"\u23cf"
                u"\u23e9"
                u"\u231a"
                u"\ufe0f"  # dingbats
                u"\u3030"
                                            "]+", re.UNICODE)

    # remove emojis
    cleaned = re.sub(emoj, '', data).strip()
    
    # convert german umlauts before removing diacritics
    cleaned = cleaned.replace('Ü','Ue').replace('Ä','Ae').replace('Ö', 'Oe').replace('ü', 'ue').replace('ä', 'ae').replace('ö', 'oe')
    
    # convert semicolon to colon to prevent CSV breaking
    cleaned = cleaned.replace(',', '')
    cleaned = cleaned.replace(';', ',')
    
    # remove diacritics
    cleaned = unidecode.unidecode(cleaned)
    
    return cleaned.strip()

## This is the first function that gives us the first record for the dataframe and contains the information of the target_user that we want to extract the contacts from

In [3]:
def employee_information(employee):
     
        try: 
            
            account_name = clean_data(employee["title"]["text"]).split(" ") #text_normalization
            badwords = ['Prof.', 'Dr.', 'M.A.', ',', 'LL.M.'] #text_normalization
            
            #text_normalization
            
            for word in list(account_name):
                if word in badwords:
                    account_name.remove(word)
            account_name = " ".join(account_name)
            
        except:
            
             pass
        
        
        # Saerch for the position 
        try:
            position = clean_data(employee["primarySubtitle"]["text"])
        except:
            position = "N/A"

        # Search for the location
        try: 
            location = employee["secondarySubtitle"]["text"]
        except:
            location = "N/A"

        # Search for the profileLink
        try:
            profile_link = employee["navigationUrl"].split("?")[0]
            profile_link = unquote(profile_link)
        except:
            profile_link = "N/A"

        
        # Search the connection_degree
        try:
            degree_str = employee["entityCustomTrackingInfo"]["memberDistance"]
            degree = int(degree_str.split("DISTANCE_")[-1])
        except:
            degree = 0
            
        
        # Search the uniqueID
        try:
            
            contact_id_urn = employee['image']['attributes'][0]['detailData']['nonEntityProfilePicture']['profile']['entityUrn'].split(":")[-1]
        
        except: 
            
            contact_id_urn = "N/A"
            
        
        # Search the userCode
        # Name of the LinkedIn account. 
        # e.x: https://www.linkedin.com/in/XXXX/ >> employee_information(XXXX)
        
        try:
            contact_userCode = employee["navigationUrl"].split("?")[0].split("/")[-1]
            contact_userCode = unquote(contact_userCode)
            
        except:
            contact_userCode = "N/A" 
            
            
        ## return a dictionary with all the user's information
        return {"userID": contact_id_urn,"userCode": contact_userCode,"Nombre":account_name, "Puesto": position, "Grado":degree, "Ubicacion":location, "Link":profile_link}


## Information of the contacts of the Target
    WE search and extract a list with the information for each of the contacts of our target_contact

In [4]:
# li_at <- Cookie information about the active session
# JSESSIONID <- Cookie information about the active session
# user <- Name of the LinkedIn account. 
# e.x: https://www.linkedin.com/in/XXXX/ >> employee_information(XXXX)

def contacts(user, li_at, JSESSIONID):
    
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/122.0.0.0 Safari/537.36 Edg/122.0.0.0', 
        'Content-type': 'application/json', 
        'Csrf-Token': JSESSIONID,
        'Cache-Control': 'max-age=0',
        'Connection': 'keep-alive'
    }
    cookies_dict = {"li_at": li_at, "JSESSIONID": JSESSIONID}
    cod_user = quote(user)
    
    
    # Get the information of the target
    url = f"https://www.linkedin.com/voyager/api/graphql?variables=(vanityName:{cod_user})&queryId=voyagerIdentityDashProfiles.d8946426aeb23ab919d181f179a67a4d"
    response = requests.get(url, headers=headers, cookies=cookies_dict)
    r = response.json()
    
    
    # Information of the target
    user_info = r['data']['identityDashProfilesByMemberIdentity']['elements'][0]
    id_urn = user_info['entityUrn'].split(":")[-1]
    name = user_info['firstName'] + " " + user_info['lastName']
    position = user_info['headline']
    location = user_info['geoLocation']['geo']['defaultLocalizedName']
    
    #create a df with the information and initialize a list for the contacts
    user_info = pd.DataFrame({"userID": [id_urn],"userCode": [cod_user],"Nombre": [name], "Puesto": [position], "Grado": [0], "Ubicacion": [location], "Link": ["https://www.linkedin.com/in/" + user]})
    list_contacts = []
    
          
    # get all the contacts iterating witha step of 3 that is the max return of the call and append the information in the list 
    
    for i in range(0,500,3):
        url = f"https://www.linkedin.com/voyager/api/graphql?variables=(start:{i},origin:MEMBER_PROFILE_CANNED_SEARCH,query:(flagshipSearchIntent:SEARCH_SRP,queryParameters:List((key:connectionOf,value:List({id_urn})),(key:network,value:List(F,S,O)),(key:resultType,value:List(PEOPLE))),includeFiltersInResponse:false))&queryId=voyagerSearchDashClusters.cc5c3924cc0402d1d8838b15bc96aa0b"
        response = requests.get(url, headers=headers, cookies=cookies_dict)
        r = response.json()
        
        # appending the information of each of the contacts
        try:
            list_users = r['data']['searchDashClustersByAll']['elements'][0]['items']
            
            for user in list_users:
                list_contacts.append(employee_information(user["item"]['entityResult']))
                
        except:
            
            pass
    
    
    #Merge the information of the target and his contacts into a unique dataframe the targets information will be the first row with a zero degree connection with himself
    contacts = pd.DataFrame(list_contacts)
    contacts = contacts.drop_duplicates()
    contacts = contacts[contacts['Nombre'] != "LinkedIn Member"]
    contacts = pd.concat([user_info, contacts], ignore_index=True)
    
    return contacts